In [5]:
import torch
import torch.optim as optim
from src.pytorch_efficientnet import EfficientNetwork, augment, augmented_pred
from src.train_test_functions import train_model
import json
import numpy as np
from src.torch_dataset import ZootrDataset 
import os
import pandas as pd
from sklearn.metrics import roc_auc_score

## Set constants

In [6]:
BATCH_SIZE = 20
NUM_WORKERS = 0
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.1
EPOCHS = 10

In [7]:
with open('config/paths_dict.json') as handle:
    paths_dict = json.loads(handle.read())

ITEM_PATH = paths_dict['item_path']
NON_ITEM_PATH = paths_dict['non_item_path']

In [8]:
load_data_objects = False
save_data_objects = True

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set up data loaders

In [10]:
if  load_data_objects:
    torch.load('Models/train_data.pth')
    torch.load('Models/valid_data.pth')
else:
    items_images = [ITEM_PATH + i for i in os.listdir(ITEM_PATH)]
    #Get all items
    non_items_images = [NON_ITEM_PATH + i for i in os.listdir(NON_ITEM_PATH)]
    #Get non items
    all_images = list(np.random.permutation(items_images + non_items_images))
    #Combine all images' paths and shuffle
    train_data = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augment)
    train_data_eval = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augmented_pred)
    valid_data = ZootrDataset(all_images[int(np.ceil(len(all_images)*0.8)):], transform_=augmented_pred)
    # Data Objects
    if save_data_objects:
        torch.save(train_data, 'Models/train_data.pth')
        torch.save(valid_data, 'Models/valid_data.pth')

In [11]:
data = {'train': all_images[0:int(np.ceil(len(all_images)*0.7))], 
        'val': all_images[int(np.ceil(len(all_images)*0.7)):int(np.ceil(len(all_images)*0.8))], 
        'test': all_images[int(np.ceil(len(all_images)*0.8)):]}
transformations = {'train': augment, 'val': augmented_pred, 'test': augmented_pred}
dataset_sizes = {'train': int(np.ceil(len(all_images)*0.7)), 'val': int(np.ceil(len(all_images)*0.8)) - int(np.ceil(len(all_images)*0.7)), 
                 'test': len(all_images) - int(np.ceil(len(all_images)*0.8))}

datasets = {x: ZootrDataset(data[x], transform_=transformations[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, drop_last=True, shuffle=False) for x in ['train', 'val', 'test']}

dics = {
    'dataloaders': dataloaders,
    'dataset_sizes': dataset_sizes
}

## Model

In [12]:
model = EfficientNetwork(output_size=2, b1=False, b2=True)
# Criterion
criterion = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                                       patience=1, factor=0.4)

Loaded pretrained weights for efficientnet-b2


In [14]:
mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=25)

Epoch 1


100%|██████████| 10/10 [00:06<00:00,  1.53it/s]


Train loss: 0.3957583849481162 Val loss: 0.278946564393343
Epoch 2


100%|██████████| 10/10 [00:06<00:00,  1.63it/s]


Train loss: 0.14366170467387912 Val loss: 0.15841740269015953
Epoch 3


...

Train loss: 0.0018121797128280582 Val loss: 0.06686816608380293
Epoch 24


100%|██████████| 10/10 [00:07<00:00,  1.28it/s]


Train loss: 0.0014293681305293162 Val loss: 0.07610421169972147
Epoch 25


100%|██████████| 10/10 [00:06<00:00,  1.47it/s]

Train loss: 0.001204128065444986 Val loss: 0.08625412473437075
Training complete in 37m 13s
Best val loss: 0.001204


## Get test metrics

In [15]:
test_loader = dataloaders['test']

all_preds = torch.empty(0, 2)
all_labels = []
for imgs, label in test_loader:
    imgs_pred = mdl(imgs)

    all_preds = torch.cat((all_preds, imgs_pred))
    all_labels += label
labels = [i.item() for i in all_labels]
#Get all labels

In [16]:
preds_prob = torch.softmax(all_preds, dim=1)
preds_prob = pd.DataFrame(preds_prob.detach().numpy())
preds_prob.columns = ['False', 'True']
preds_prob['Label'] = labels

In [17]:
preds_prob.sort_values('True', ascending=False)

,False,True,Label
82,0.000009,9.999911e-01,1
365,0.000036,9.999641e-01,1
319,0.000060,9.999399e-01,1
343,0.000065,9.999348e-01,1
44,0.000079,9.999208e-01,1
...,...,...,...
132,0.999998,1.600326e-06,0
108,0.999999,9.364485e-07,0
6,0.999999,8.264178e-07,0
282,0.999999,7.576942e-07,0


In [18]:
roc_auc_score(preds_prob['Label'], preds_prob['True'])

0.9696310890685508

In [19]:
1/0

ZeroDivisionError: division by zero

## MLflow

In [20]:
import mlflow

In [21]:
!mlflow ui --backend-store-uri file:Tracking
#Needs to be interrupted

^C


In [22]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("check-localhost-connection")

<Experiment: artifact_location='mlflow-artifacts:/664169674059468358', creation_time=1755030492211, experiment_id='664169674059468358', last_update_time=1755030492211, lifecycle_stage='active', name='check-localhost-connection', tags={}>

In [ ]:
NR_EPOCHS=5

with mlflow.start_run() as run:
  params = {
      "num_epochs": NR_EPOCHS,
      "scheduler": scheduler.__dict__,
      "optimizer": optimizer
  }
  # Log training parameters.
  mlflow.log_params(params)

  # Log model summary.
  #with open("model_summary.txt", "w") as f:
  #    f.write(str(summary(model)))
  #mlflow.log_artifact("model_summary.txt")

  model = EfficientNetwork(output_size=2, b1=False, b2=True)
  mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=NR_EPOCHS)
  

  # Save the trained model to MLflow.
  model_info = mlflow.pytorch.log_model(mdl, name="first_model")

  all_preds = torch.empty(0, 2)
  all_labels = []
  for imgs, label in test_loader:
      imgs_pred = mdl(imgs)

      all_preds = torch.cat((all_preds, imgs_pred))
      all_labels += label
  labels = [i.item() for i in all_labels]

  preds_prob = torch.softmax(all_preds, dim=1)
  preds_prob = pd.DataFrame(preds_prob.detach().numpy())
  preds_prob.columns = ['False', 'True']
  preds_prob['Label'] = labels

  test_loss = criterion(all_preds, torch.tensor(labels)).item()
  test_roc = roc_auc_score(preds_prob['Label'], preds_prob['True'])

  mlflow.log_metrics({"test_loss": test_loss, "test_roc": test_roc})

Loaded pretrained weights for efficientnet-b2
Epoch 1


100%|██████████| 10/10 [00:06<00:00,  1.65it/s]


Train loss: 0.6719619039840012 Val loss: 0.7109224047637792
Epoch 2


100%|██████████| 10/10 [00:06<00:00,  1.64it/s]


Train loss: 0.6692260170114724 Val loss: 0.7056232523802973
Epoch 3


...

Train loss: 0.6718063230916559 Val loss: 0.6922866987145465
Epoch 5


100%|██████████| 10/10 [00:05<00:00,  1.69it/s]


Train loss: 0.6713295378352168 Val loss: 0.6915592628976573
Training complete in 4m 14s
Best val loss: 0.668954


2025/08/12 22:42:02 WARNING mlflow.utils.requirements_utils: Found torch version (2.7.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.7.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/08/12 22:42:14 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.22.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.22.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/08/12 22:42:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run learned-sow-145 at: http://localhost:5000/#/experiments/664169674059468358/runs/c0e049ac9479444fb294bcb1372b975d
🧪 View experiment at: http://localhost:5000/#/experiments/664169674059468358


In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)